# Completion APIs With Custom Data
This Jupyter notebook enables simple interactions with LLM APIs. The various APIs are quite similar and this notebook can be extended with relative ease. The code can be integrated with other means to retrieve input data, such as separate PDF, Word documents or third-party APIs.

## Setup
The notebook below has been run with Python3.11. It requires the following modules to be installed:
* `openai`: The OpenAI Python library to communicate with the OpenAI API, such as the GPT-4 LLM
* `groq`: The Groq Python library to communicate with the Groq API and the open source models they host, such as Llama 3.

In [57]:
import os
from groq import Groq
from openai import OpenAI

In [58]:
from typing import Dict, List, Optional

The APIs require authentication. A personal API key has to be created by the user. That can be done at:
* https://console.groq.com/keys
* https://platform.openai.com/api-keys

It is good safety practice to store the keys in an environment variable and access the key in the code below via said environment variable. The keys will be assumed to be stored in `GROQ_API_KEY` and `OPENAI_API_KEY`. Do not share the API keys.

**Note**: Groq API can be used for free (for now) though with rate limits. It is a good way to experiement with code, which can be applied to other LLMs.

In [59]:
%env GROQ_API_KEY=<insert KEY>

env: GROQ_API_KEY=<insert KEY>


Convenience functions are defined for accessing the LLM APIs.

In [60]:
def get_openai_client() -> OpenAI:
    return OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def get_groq_client() -> Groq:
    return Groq(api_key=os.environ.get('GROQ_API_KEY'))

In [61]:
def send_request_to_openai_chat_completion(
    client: OpenAI,
    messages: List[Dict[str, str]],
    model: str = 'gpt-4-turbo-2024-04-09',
    tools: Optional[List[str]] = None,
    tool_choice: str = 'auto',
    temperature: float = 0.7,
    max_tokens: int = 4096,
    top_p: float = 1.0,
    frequency_penalty: float = 0.0,
    presence_penalty: float = 0.0,
    n_completions: int = 1,
):
    kwargs = {
        'model': model,
        'messages': messages,
        'tools': tools,
        'tool_choice': tool_choice,
        'temperature': temperature,
        'top_p': top_p,
        'max_tokens': max_tokens,
        'frequency_penalty': frequency_penalty,
        'presence_penalty': presence_penalty,
        'n_completions': n_completions
    }
    if tools is None:
        del kwargs['tools']
        del kwargs['tool_choice']
        
    try:
        openai_completion = client.chat.completions.create(**kwargs)
    except Exception as e:
        raise e
        
    return openai_completion


def send_request_to_groq_chat_completion(
    client: Groq,
    messages: List[Dict[str, str]],
    model: str = 'llama3-8b-8192',
    temperature: float = 0.7,
    top_p: float = 1.0,
    max_tokens: int = 8192
):
    kwargs = {
        'model': model,
        'messages': messages,
        'temperature': temperature,
        'top_p': top_p,
        'max_tokens': max_tokens,
    }
    
    try:
        groq_completion = client.chat.completions.create(**kwargs)
    except Exception as e:
        raise e
        
    return groq_completion

The LLM can process several messages. This is useful when a chat with memory is desired. However, to pass old messages to the LLM each time uses up more input tokens. When memory is not required, it is therefore good to delete old messages. The `MessageStack` class defines methods to abstract how the text is given as input to the LLMs.

In [62]:
class MessageStack:
    def __init__(self):
        self._message_collection = []
        
    def __repr__(self):
        ret_str = ''
        for row in self._message_collection:
            ret_str += f'{row["role"]}: {row["content"]}\n'
            ret_str += '=' * 50
            ret_str += '\n'
        return ret_str[:-51]
        
    def add(self, role: str, content: str):
        if not role in ['system', 'user', 'assistant']:
            raise ValueError(f'Invalid role {role} encountered.')
            
        self._message_collection.append({'role': role, 'content': content})
        
    def set_system_instruction(self, value: str):
        for row in self._message_collection:
            if row['role'] == 'system':
                row['content'] = value
                break
        else:
            self.add('system', value)
        
    def add_user_content(self, value: str):
        self.add('user', value)
        
    def get_message_stack(self):
        return self._message_collection
    
    def delete(self, slc):
        del self._message_collection[slc]
        

## Test Run
A few short test runs are provided below as illustration.

In [47]:
groq_client = get_groq_client()

In [54]:
message_stack = MessageStack()
message_stack.set_system_instruction('You are a minimalist poet')
message_stack.add_user_content('Please write a haiku about quantum mechanics')
print (message_stack)

system: You are a minimalist poet
user: Please write a haiku about quantum mechanics



In [49]:
out = send_request_to_groq_chat_completion(client=groq_client, messages=message_stack.get_message_stack())
print (out.choices[0].message.content)

Particles spin slow
Uncertainty's dark veil
Reality's haze


In [50]:
out = send_request_to_groq_chat_completion(client=groq_client, messages=message_stack.get_message_stack(), model='llama3-70b-8192')
print (out.choices[0].message.content)

Whispers of chance dance
Probabilities entwined
Uncertainty


In [55]:
message_stack.set_system_instruction('You are a geographer, erudite and wise, and willing to assist')
message_stack.delete(-1)
message_stack.add_user_content('What is the difference between Scandinavia and the Nordic countries? Seem pretty similar.')
print (message_stack)

system: You are a geographer, erudite and wise, and willing to assist
user: What is the difference between Scandinavia and the Nordic countries? Seem pretty similar.



In [56]:
out = send_request_to_groq_chat_completion(client=groq_client, messages=message_stack.get_message_stack(), model='llama3-70b-8192')
print (out.choices[0].message.content)

An excellent question, my curious friend! Many people do indeed use the terms "Scandinavia" and "Nordic countries" interchangeably, but there is a subtle distinction between them.

Scandinavia refers specifically to the region that comprises Denmark, Norway, and Sweden. These three countries share cultural, linguistic, and historical ties, including a common North Germanic heritage and similar languages (Danish, Norwegian, and Swedish). The term "Scandinavia" is derived from the Scandinavian Mountains, which run through Norway and Sweden.

On the other hand, the Nordic countries, also known as the Nordics, encompass a broader region that includes not only Denmark, Norway, and Sweden but also Finland, Iceland, the Faroe Islands, Greenland, and the Åland Islands. This grouping is often referred to as Norden.

The Nordic countries share a common cultural and historical heritage, as well as a similar economic and social model, often referred to as the "Nordic model." This model is characte